# Random Forest
Train on 6-class cancer classification.  
Input is the nucleus+RBC rollup, training set only.  
Train on the full training set (no validation).  
Rank feature importance.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-06-09 09:16:41.719930
Python 3.8.10
sklearn 1.0.2


[]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='/home/jrm/Adjeroh/Naved/CP_80K/'  # append Output0/ etc
NUCLEUS_FILES=[
    'Nucleus_Rollup_0.csv',
    'Nucleus_Rollup_1.csv',
    'Nucleus_Rollup_2.csv',
    'Nucleus_Rollup_3.csv',
    'Nucleus_Rollup_4.csv',
    'Nucleus_Rollup_5.csv']
RBC_FILES=[
    'RBC_Rollup_0.csv',
    'RBC_Rollup_1.csv',
    'RBC_Rollup_2.csv',
    'RBC_Rollup_3.csv',
    'RBC_Rollup_4.csv',
    'RBC_Rollup_5.csv']
CLASSES=range(0,6)  # use all 6 classes
CLASSES=[0]  # JUST FOR TESTING
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.17/'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Process class",i)
        FULL_PATH=BASE_DIR+'Output'+str(i)+'/'
        cp = CP_Util(FULL_PATH)
        cp.train_test_split()
        Xnuc = cp.get_nucleus_rollup(NUCLEUS_FILES[i])
        print('Xnuc size',Xnuc.shape)
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = cp.get_RBC_rollup(RBC_FILES[i])
        print('Xrbc size',Xrbc.shape)
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-09 09:16:43.810634
2022-06-09 09:16:43.810832
Process class 0
object_info: Int64Index([  397,   398,   399,   400,   401,   402,   403,   404,   405,
              406,
            ...
            44957, 44958, 44959, 44960, 44961, 44962, 44963, 44964, 44965,
            44966],
           dtype='int64', name='PatchNumber', length=35950)
good_patches: Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            43444, 43445, 43446, 43447, 43448, 43449, 43450, 43451, 43452,
            43453],
           dtype='int64', name='PatchNumber', length=8803)
Xnuc size (0, 5192)
object_info: Int64Index([  403,   413,   421,   422,   424,   428,   442,   446,   449,
              455,
            ...
            44858, 44859, 44860, 44866, 44867, 44884, 44915, 44917, 44937,
            44952],
           dtype='int64', name='PatchNumber', length=8476)
good_patches: Int64Index([    1,     2,     3,     4,     5,     6,     7,   

,Nuc_ObjectNumber_count,Nuc_ObjectNumber_mean,Nuc_ObjectNumber_std,Nuc_ObjectNumber_min,Nuc_ObjectNumber_25%,Nuc_ObjectNumber_50%,Nuc_ObjectNumber_75%,Nuc_ObjectNumber_max,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,...,Rbc_Number_Object_Number_75%,Rbc_Number_Object_Number_max,Rbc_Parent_ShrinkRBC_count,Rbc_Parent_ShrinkRBC_mean,Rbc_Parent_ShrinkRBC_std,Rbc_Parent_ShrinkRBC_min,Rbc_Parent_ShrinkRBC_25%,Rbc_Parent_ShrinkRBC_50%,Rbc_Parent_ShrinkRBC_75%,Rbc_Parent_ShrinkRBC_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,


In [5]:
print(datetime.datetime.now())
print("Shuffle...")
# Shuffle (happens automatically with cross validation but not with fit())
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-09 09:17:21.423094
Shuffle...


In [6]:
print(datetime.datetime.now())
print('Train on full training set...')
rf3 = RF_Util()
rf3.set_train(Xtrain,ytrain)
rf3.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf3.important_features()

2022-06-09 09:17:21.426930
Train on full training set...


ValueError: Found array with 0 sample(s) (shape=(0, 6392)) while a minimum of 1 is required.

In [ ]:
pd.set_option('display.max_rows', None)
top.loc[:100]

In [ ]:
print(datetime.datetime.now())
joblib.dump(rf3.get_model(),MODELS_DIR)